In [63]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Loading libraries
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import pairwise_distances
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import re
import seaborn as sns
data1=pd.read_csv('phone_user_review_file_1.csv',engine='python')
data2=pd.read_csv('phone_user_review_file_2.csv',engine='python')
data3=pd.read_csv('phone_user_review_file_3.csv',engine='python')
data4=pd.read_csv('phone_user_review_file_4.csv',engine='python')
data5=pd.read_csv('phone_user_review_file_5.csv',engine='python')
data6=pd.read_csv('phone_user_review_file_6.csv',engine='python')

In [64]:
# Merge all datasets onto one and explore final shape and size.
df=[data1, data2, data3, data4, data5, data6]

Data_Final = pd.concat(df,axis=0,ignore_index=False)
Data_Final.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [65]:
Data_Final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


In [66]:
Data_Final.shape
data6.shape

(1415133, 11)

(163837, 11)

In [67]:
#round off the scores to the nearest integer
decimals = pd.Series([0], index=['score'])
de=Data_Final.round(decimals).head()
de.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [68]:
# Check for missing value
Data_Final.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [69]:
#extract column we will drop because we dont need
#we wil delete missing values in author
#and impute the missing values in score and scoremx values
Data_Final['score'].fillna((Data_Final['score'].median()), inplace=True)
Data_Final['score'] = Data_Final['score'].astype('float')
Data_Final['score_max'].fillna((Data_Final['score_max'].median()), inplace=True)
Data_Final['score_max'] = Data_Final['score_max'].astype('float')


In [70]:

df=Data_Final.drop(columns=['extract'])
df.dropna(subset=['author'])

,phone_url,date,lang,country,source,domain,score,score_max,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...
163832,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,david.paul,Alcatel Club Plus Handy
163833,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Christiane14,Alcatel Club Plus Handy
163834,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,michaelawr,Alcatel Club Plus Handy
163835,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,claudia0815,Alcatel Club Plus Handy


In [71]:
#duplicate rows are delelted
df6 = df.drop_duplicates()
df6.head()
df6.shape

,phone_url,date,lang,country,source,domain,score,score_max,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,S Ate Mine,Samsung Galaxy S8


(1341990, 10)

In [72]:
#selecting 1000000 random samples from the dataset
df6.sample(n=1000000,random_state=612,axis=0)

,phone_url,date,lang,country,source,domain,score,score_max,author,product
10701,/cellphones/samsung-galaxy-s7-edge/,4/16/2016,en,us,Samsung,samsung.com,9.6,10.0,Nohonoel,Samsung Galaxy S7 edge 32GB (T-Mobile)
35728,/cellphones/samsung-galaxy-s7-789999/,7/23/2016,de,de,Amazon,amazon.de,6.0,10.0,unimor,"Samsung Galaxy S7 Smartphone (5,1 Zoll (12,9 c..."
310069,/cellphones/samsung-galaxy-note-edge/,12/16/2014,en,us,Amazon,amazon.com,6.0,10.0,B. J. Swanson,Samsung Galaxy Note Edge N915G 32GB Unlocked G...
28498,/cellphones/samsung-galaxy-s7-789999/,4/24/2016,en,us,Samsung,samsung.com,10.0,10.0,Riyka,Samsung Galaxy S7 32GB (Sprint)
86448,/cellphones/samsung-galaxy-s6-edge-sm-g925f/,4/11/2015,nl,nl,KIESKEURIG,kieskeurig.nl,9.2,10.0,Noener,Samsung Galaxy S6 edge zwart / 32 GB
...,...,...,...,...,...,...,...,...,...,...
266845,/cellphones/karbonn-a9/,11/29/2012,en,in,Naaptol,naaptol.com,10.0,10.0,VIKRAM GURVE,Karbonn A9
53978,/cellphones/apple-iphone-5s/,1/15/2014,ru,ru,Ð¡Ð²Ñ�Ð·Ð½Ð¾Ð¹,svyaznoy.ru,10.0,10.0,Ð•Ð²Ð³ÐµÐ½Ð¸Ð¹ Ð§ÐµÑ€ÐºÐ°Ñ�Ð¾Ð²,Apple iPhone 5s 16GB (Ñ�ÐµÑ€ÐµÐ±Ñ€Ð¸Ñ�Ñ‚Ñ‹Ð¹)
298741,/cellphones/nokia-lumia-920/,10/25/2013,ru,ua,Rozetka,rozetka.com.ua,8.0,10.0,Lotus_Esprit,Nokia Lumia 920 White
162491,/cellphones/blackberry-9700/,6/1/2010,pt,br,MercadoLibre,opiniao.mercadolivre.com.br,10.0,10.0,RAPTORBSB,BlackBerry 9700


In [74]:
#now we have to identify the most rated features
import pandas as pd
import numpy as np
de['rank']=de.apply(lambda row: row.score,axis=1)
print(de)

                        phone_url       date lang country            source  \
0  /cellphones/samsung-galaxy-s8/   5/2/2017   en      us  Verizon Wireless   
1  /cellphones/samsung-galaxy-s8/  4/28/2017   en      us       Phone Arena   
2  /cellphones/samsung-galaxy-s8/   5/4/2017   en      us            Amazon   
3  /cellphones/samsung-galaxy-s8/   5/2/2017   en      us           Samsung   
4  /cellphones/samsung-galaxy-s8/  5/11/2017   en      us  Verizon Wireless   

                domain  score  score_max  \
0  verizonwireless.com   10.0       10.0   
1       phonearena.com   10.0       10.0   
2           amazon.com    6.0       10.0   
3          samsung.com    9.0       10.0   
4  verizonwireless.com    4.0       10.0   

                                             extract       author  \
0  As a diehard Samsung fan who has had every Sam...   CarolAnn35   
1  Love the phone. the phone is sleek and smooth ...    james0923   
2  Adequate feel. Nice heft. Processor's still sl... 

In [75]:
#now we have to rank the table 
de.rank().head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product,rank
0,3.0,3.5,3.0,3.0,4.5,4.5,4.5,3.0,2.0,2.0,2.0,4.5
1,3.0,1.0,3.0,3.0,2.0,2.0,4.5,3.0,4.0,5.0,2.0,4.5
2,3.0,5.0,3.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,4.0,2.0
3,3.0,3.5,3.0,3.0,3.0,3.0,3.0,3.0,5.0,1.0,5.0,3.0
4,3.0,2.0,3.0,3.0,4.5,4.5,1.0,3.0,3.0,4.0,2.0,1.0


In [76]:
de['rank_by_score']=de['rank'].rank(ascending=False)
de.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product,rank,rank_by_score
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8,10.0,1.5
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8,10.0,1.5
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",6.0,4.0
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T),9.0,3.0
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8,4.0,5.0


In [77]:
#sort the ranking colums
de.sort_values('rank',ascending=False)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product,rank,rank_by_score
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8,10.0,1.5
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8,10.0,1.5
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T),9.0,3.0
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",6.0,4.0
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8,4.0,5.0


In [82]:
#the dataset is sorted based on rank
#we can see the most rated feature product is samsung galaxys8
#now we have to identify the author with the most number of reviews.
count = Data_Final['author'].value_counts() 
print(count)

Amazon Customer                         76978
Cliente Amazon                          19304
e-bit                                    8663
Client d'Amazon                          7716
Amazon Kunde                             4750
                                        ...  
Cristina Navarro                            1
Antonio Fabiano Santos Do Nascimento        1
bafiazzo                                    1
Joseph Spiteri                              1
PADMAGANAPATHY...                           1
Name: author, Length: 801098, dtype: int64


In [236]:
#as we can see the amazon customer has given the most number of reviews followed by the cliente amazon
#now coming to part three
Data_Final.groupby('product')['score'].count().sort_values(ascending=False).head()

product
Lenovo Vibe K4 Note (White,16GB)     5226
Lenovo Vibe K4 Note (Black, 16GB)    4390
OnePlus 3 (Graphite, 64 GB)          4103
OnePlus 3 (Soft Gold, 64 GB)         3563
Huawei P8lite zwart / 16 GB          2707
Name: score, dtype: int64

In [103]:
#now coming to the popularity recommendation system for top 5 mobile phones
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df6, test_size = 0.20, random_state=0)
print(train_data.head(5))

                                phone_url        date lang country  \
306544  /cellphones/samsung-galaxy-s-iii/  10/19/2013   en      us   
43799             /cellphones/nokia-1208/    4/4/2009   de      de   
127841           /cellphones/philips-568/   7/15/2009   fr      fr   
217709         /cellphones/samsung-b3410/   5/25/2010   en      us   
54632    /cellphones/lenovo-vibe-k4-note/    2/6/2016   en      in   

             source           domain  score  score_max          author  \
306544  Phone Arena   phonearena.com    9.0       10.0          RavX96   
43799        Amazon        amazon.de    8.0       10.0          Stegel   
127841  GraphMobile  graphmobile.com    8.0       10.0    Anani HOUNZA   
217709  Phone Arena   phonearena.com    1.0       10.0  Santosh Matkar   
54632        Amazon        amazon.in   10.0       10.0       prashanth   

                                                  product  
306544                               Samsung Galaxy S III  
43799   Nokia 

In [104]:
#Count of user_id for each unique mobile as recommendation score 
train_data_grouped = train_data.groupby('product').agg({'author': 'count'}).reset_index()
train_data_grouped.rename(columns = {'author': 'score'},inplace=True)
train_data_grouped.head()

,product,score
0,'Sony Xperia X (F5122) â€“ rosa â€“ Dual Sim (...,1
1,"(7.62 cm (3 )Afficheur/Ã©cran, 2 MPixCamÃ©ra;b...",1
2,"(CUBOT) GT88 5.5"" qHD 1.3GHz MTK6572 2-Core An...",1
3,(DG300 Versione Aggiornata)5'' DOOGEE VOYAGER2...,39
4,(Part 2) Lenovo VIBE X2,1


In [105]:
#Sort the songs on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'product'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations 

,product,score,Rank
16210,Huawei P8lite zwart / 16 GB,2177,1.0
32927,"OnePlus 3 (Graphite, 64 GB)",1926,2.0
42317,Samsung Galaxy S6 zwart / 32 GB,1888,3.0
38844,Samsung Galaxy Express I8730,1852,4.0
22430,"Lenovo Vibe K4 Note (White,16GB)",1837,5.0


In [112]:
#now to build a collaborative filtering model using svd
dfb = df6[df6.phone_url.isin(df6.phone_url)]
di = dfb[dfb.score!= 0]
counts1 = di['author'].value_counts()
di = di[di['author'].isin(counts1[counts1 >= 75].index)]


In [113]:
di.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product
55,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Black"
66,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey"
68,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Orchid Grey"
106,/cellphones/samsung-galaxy-s8/,5/5/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey"
161,/cellphones/samsung-galaxy-s8/,4/30/2017,en,gb,Very,very.co.uk,10.0,10.0,Paul,"Samsung Galaxy S8, 64Gb - Black"


In [116]:
dr=di.drop_duplicates(keep='first', inplace=False)
dr.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product
55,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Black"
66,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey"
68,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Orchid Grey"
106,/cellphones/samsung-galaxy-s8/,5/5/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey"
161,/cellphones/samsung-galaxy-s8/,4/30/2017,en,gb,Very,very.co.uk,10.0,10.0,Paul,"Samsung Galaxy S8, 64Gb - Black"


In [127]:
#now lets add another column giving each author a new user id
import numpy as np
dr['user_id'] = dr.reset_index().index 
dr.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,user_id
55,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Black",0
66,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey",1
68,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Orchid Grey",2
106,/cellphones/samsung-galaxy-s8/,5/5/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey",3
161,/cellphones/samsung-galaxy-s8/,4/30/2017,en,gb,Very,very.co.uk,10.0,10.0,Paul,"Samsung Galaxy S8, 64Gb - Black",4


In [128]:

ratings_matrix = dr.pivot(index='user_id', columns='phone_url', values='score').fillna(0)
user_id = ratings_matrix.index
phone_url = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix.head()

(171603, 3834)


phone_url,/cellphones/acer-acer-liquid-m220/,/cellphones/acer-betouch-e100/,/cellphones/acer-betouch-e101/,/cellphones/acer-betouch-e110/,/cellphones/acer-betouch-e130/,/cellphones/acer-betouch-e140/,/cellphones/acer-betouch-e200/,/cellphones/acer-betouch-e210/,/cellphones/acer-betouch-e400/,/cellphones/acer-cloudmobile/,...,/cellphones/zte-z431/,/cellphones/zte-z432/,/cellphones/zte-z5/,/cellphones/zte-z992/,/cellphones/zte-z998/,/cellphones/zte-zinger/,/cellphones/zte-zmax-2/,/cellphones/zte-zmax/,/cellphones/zte-zte-g-r221/,/cellphones/zte-zte-u-f930/
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_matrix, k = 50)

In [130]:
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = ratings_matrix.columns)

In [131]:
preds_df.head()

phone_url,/cellphones/acer-acer-liquid-m220/,/cellphones/acer-betouch-e100/,/cellphones/acer-betouch-e101/,/cellphones/acer-betouch-e110/,/cellphones/acer-betouch-e130/,/cellphones/acer-betouch-e140/,/cellphones/acer-betouch-e200/,/cellphones/acer-betouch-e210/,/cellphones/acer-betouch-e400/,/cellphones/acer-cloudmobile/,...,/cellphones/zte-z431/,/cellphones/zte-z432/,/cellphones/zte-z5/,/cellphones/zte-z992/,/cellphones/zte-z998/,/cellphones/zte-zinger/,/cellphones/zte-zmax-2/,/cellphones/zte-zmax/,/cellphones/zte-zte-g-r221/,/cellphones/zte-zte-u-f930/
0,-9.756048e-33,-2.001699e-35,2.559517e-33,5.671409e-33,-1.357857e-33,7.745058e-34,-2.476447e-33,2.469667e-33,-2.123038e-33,-1.219281e-32,...,2.239801e-32,1.082221e-32,-4.654045e-34,-2.267548e-32,2.568992e-33,-8.714355e-33,1.655443e-32,-1.752304e-33,8.219730e-33,-5.282843e-33
1,-9.756048e-33,-2.001699e-35,2.559517e-33,5.671409e-33,-1.357857e-33,7.745058e-34,-2.476447e-33,2.469667e-33,-2.123038e-33,-1.219281e-32,...,2.239801e-32,1.082221e-32,-4.654045e-34,-2.267548e-32,2.568992e-33,-8.714355e-33,1.655443e-32,-1.752304e-33,8.219730e-33,-5.282843e-33
2,-9.756048e-33,-2.001699e-35,2.559517e-33,5.671409e-33,-1.357857e-33,7.745058e-34,-2.476447e-33,2.469667e-33,-2.123038e-33,-1.219281e-32,...,2.239801e-32,1.082221e-32,-4.654045e-34,-2.267548e-32,2.568992e-33,-8.714355e-33,1.655443e-32,-1.752304e-33,8.219730e-33,-5.282843e-33
3,-9.756048e-33,-2.001699e-35,2.559517e-33,5.671409e-33,-1.357857e-33,7.745058e-34,-2.476447e-33,2.469667e-33,-2.123038e-33,-1.219281e-32,...,2.239801e-32,1.082221e-32,-4.654045e-34,-2.267548e-32,2.568992e-33,-8.714355e-33,1.655443e-32,-1.752304e-33,8.219730e-33,-5.282843e-33
4,-9.756048e-33,-2.001699e-35,2.559517e-33,5.671409e-33,-1.357857e-33,7.745058e-34,-2.476447e-33,2.469667e-33,-2.123038e-33,-1.219281e-32,...,2.239801e-32,1.082221e-32,-4.654045e-34,-2.267548e-32,2.568992e-33,-8.714355e-33,1.655443e-32,-1.752304e-33,8.219730e-33,-5.282843e-33


In [133]:
userid = 2

In [134]:
user_id = ratings_matrix.iloc[userid-1, :].name
user_id

1

In [135]:
preds_df.shape

(171603, 3834)

In [136]:
sorted_user_predictions = preds_df.iloc[userid].sort_values(ascending=False)

In [137]:
len(sorted_user_predictions)

3834

In [139]:
# Get all user interacted books
user_data = dr[dr.user_id == (user_id)]

In [140]:
user_data.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,user_id
66,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey",1


In [141]:
user_data.shape

(1, 11)

In [142]:
mob_data = dr[dr.phone_url.isin(user_data.phone_url)]

In [143]:
mob_data.shape

(50, 11)

In [144]:
mob_data.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,user_id
55,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Black",0
66,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey",1
68,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,Anonymous,"Samsung Galaxy S8, 64Gb - Orchid Grey",2
106,/cellphones/samsung-galaxy-s8/,5/5/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey",3
161,/cellphones/samsung-galaxy-s8/,4/30/2017,en,gb,Very,very.co.uk,10.0,10.0,Paul,"Samsung Galaxy S8, 64Gb - Black",4


In [145]:
user_full_info = user_data.merge(mob_data)

In [146]:
user_full_info.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,user_id
0,/cellphones/samsung-galaxy-s8/,5/6/2017,en,gb,Very,very.co.uk,10.0,10.0,James,"Samsung Galaxy S8, 64Gb - Orchid Grey",1


In [151]:
recommendations = (dr[~dr['phone_url'].isin(user_full_info['phone_url'])].
                   merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'phone_url'
                         ,right_on = 'phone_url')).rename(columns = {user_id: 'Predictions'})

In [152]:
recommendations.shape

(171553, 12)

In [153]:
recommendations.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,user_id,2
0,/cellphones/samsung-galaxy-s6-edgeplus/,3/2/2017,en,us,Samsung,samsung.com,10.0,10.0,Anonymous,Samsung Galaxy S6 edge+ 32GB (T-Mobile),50,-8.892708e-31
1,/cellphones/samsung-galaxy-s6-edgeplus/,1/23/2017,en,us,Amazon,amazon.com,10.0,10.0,William,Samsung Galaxy S6 Edge Plus 32GB Unlocked Quad...,51,-8.892708e-31
2,/cellphones/samsung-galaxy-s6-edgeplus/,1/23/2017,en,us,Amazon,amazon.com,10.0,10.0,Amazon Customer,Samsung Galaxy S6 Edge Plus 32GB Unlocked Quad...,52,-8.892708e-31
3,/cellphones/samsung-galaxy-s6-edgeplus/,9/12/2016,en,us,Samsung,samsung.com,9.2,10.0,Anonymous,Samsung Galaxy S6 edge+ 64GB (Verizon),53,-8.892708e-31
4,/cellphones/samsung-galaxy-s6-edgeplus/,9/6/2016,en,us,Amazon,amazon.com,10.0,10.0,Amazon Customer,Samsung Galaxy S6 Edge+ G928G 32GB Unlocked Ph...,54,-8.892708e-31


In [155]:
#Get top 10 recommendation for above given userID
recommendations.sort_values('user_id', ascending = False).iloc[:10, :]

,phone_url,date,lang,country,source,domain,score,score_max,author,product,user_id,2
171552,/cellphones/ericsson-gf-768/,7/12/2000,en,gb,Dooyoo,dooyoo.co.uk,2.0,10.0,anonym,Ericsson GF 768,171602,4.395764e-33
171551,/cellphones/ericsson-gf-768/,7/12/2000,en,gb,Dooyoo,dooyoo.co.uk,4.0,10.0,anonym,Ericsson GF 768,171601,4.395764e-33
171550,/cellphones/motorola-startac-85/,10/14/2000,en,gb,Dooyoo,dooyoo.co.uk,8.0,10.0,anonym,Motorola Star Tac 85,171600,2.078207e-35
171549,/cellphones/philips-savvy/,12/18/2000,en,gb,Dooyoo,dooyoo.co.uk,8.0,10.0,anonym,Philips Savvy (new version),171599,5.606136e-33
171548,/cellphones/philips-savvy/,4/22/2001,en,gb,Dooyoo,dooyoo.co.uk,6.0,10.0,anonym,Philips Savvy,171598,5.606136e-33
171547,/cellphones/philips-savvy/,4/26/2001,en,gb,Dooyoo,dooyoo.co.uk,6.0,10.0,anonym,Philips Savvy,171597,5.606136e-33
171546,/cellphones/philips-savvy/,4/26/2001,en,gb,Dooyoo,dooyoo.co.uk,2.0,10.0,anonym,Philips Savvy (new version),171596,5.606136e-33
171545,/cellphones/philips-savvy/,5/18/2001,en,gb,Dooyoo,dooyoo.co.uk,4.0,10.0,anonym,Philips Savvy,171595,5.606136e-33
171544,/cellphones/philips-savvy/,9/2/2001,en,gb,Dooyoo,dooyoo.co.uk,10.0,10.0,anonym,Philips Savvy,171594,5.606136e-33
171543,/cellphones/philips-savvy/,9/28/2001,en,gb,Dooyoo,dooyoo.co.uk,6.0,10.0,anonym,Philips Savvy,171593,5.606136e-33


In [163]:
#now collaborative filtering using knn surprise

In [228]:
import surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
dw=df6[['author','product','score']]
da=dw.sample(n=5000,random_state=612,axis=0) #resampling the data
da.shape

(5000, 3)

In [221]:
#true/false statement to switch between user and item based collaborative filtering
algo= KNNWithMeans(k=50,sim_options={'name':'pearson_baseline','user_based':False})#false for item based filtering

In [229]:
#load the dataset
reader=Reader(rating_scale=(1,10))
data=Dataset.load_from_df(da,reader)
trainset,testset=train_test_split(data,test_size=0.15)

In [230]:
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [231]:
uid=str(14) #raw user from id
iid=str(30)#raw product id

In [232]:
pred=algo.predict(uid,iid,verbose=True)

user: 14         item: 30         r_ui = None   est = 8.07   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


In [233]:
#run the trained model
test_pred=algo.test(testset)

In [242]:
test_pred

[Prediction(uid='Daniel', iid='BQ Aquaris E5 HD - Smartphone libre Android (pantalla 5 pulgadas, cÃ¡mara 13 Mp, 16 GB, Quad-Core 1.3 GHz, 1 GB RAM), blanco y negro', r_ui=8.0, est=8.067505882352942, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='sijo s.', iid='Nokia 230 Dual Sim (Dark Silver)', r_ui=8.0, est=8.067505882352942, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='coldleaf', iid='LG Nexus 4 16Gb', r_ui=10.0, est=8.067505882352942, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='Amazon Customer', iid='Lenovo Vibe X2-AP (Gold)', r_ui=10.0, est=7.994732096145774, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Suraye', iid='Apple iPhone 6 Space Grau 64GB SIM-Free Smartphone (Zertifiziert und GeneralÃ¼berholt)', r_ui=10.0, est=8.067505882352942, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 P

In [238]:
accuracy.rmse(test_pred)

RMSE: 2.6253


2.625294537221121

In [239]:
#cross validation
from surprise.model_selection import cross_validate
results = cross_validate(
    algo = algo, data = data, measures=['RMSE'], 
    cv=5, return_train_measures=True
    )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [240]:
results['test_rmse'].mean()

2.592097532889674

In [243]:
#it would be more prudent to use popularity based recommenadtion system in specific cases when the user wants to know what is 'trending'
# such as in google typebox the engine automatically suggests some trending news for us to view but it is highly dependent on the users 
#interests.
#it can also be used in those business scenarios when some companies want to understand the current trending job prospcts and
#market scenarios so that they can upscale and provide competiotion in the market
#it can also display trendy fashion waear for users


In [244]:
#collaborative filtering is more user oriented approach and should definietly be used for more complex experiences for the
#recommendation systems. 
#the applications of the recommendation system is endless from ecommerce to movies tourism and other
#websites. here the model focuses more on the user integrated personal preference with trendy searches.
# hence this type of model is more generally preferred in sophisticated machines and searches
#in order to avoid the shortcomings of the prs the cf is used


In [246]:
#in order to further improve the recommendation for differnet users we can use the different other tuning techniques that can
#overcome repitition like in youtube the machine will keep recommending the same thing everyday. so to improve the recommendation 
#it should onstantly evolve to the likes of the user and constantly recieve feedback
#hence the error can be minimised and items that are viewed need not be recommended again
#we can also integrate multiple recommendation system an aplly at users will or depending on the users interest. hence then the
#user will feel satisfied with the recommendations. ultimately users should appeal to the recommendations